In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def get_url(soup):
    div_tags = soup.find_all("div", class_="item")
    links = [div_tag.find("span", class_="title").find("a")["href"] for div_tag in div_tags]
    return ["https://news.rthk.hk" + link for link in links]

def news_extract(target_url):
    news_response= session.get(target_url)
    news_soup = BeautifulSoup(news_response.content, "html.parser")
    news_title= news_soup.find("h2", class_="itemTitle").text.strip()
    news_content= news_soup.find("div",class_="itemFullText").text.strip()
    news_date= news_soup.find("div",class_="createddate").text.strip()
    return {"Title": news_title, "Content": news_content, "Date": news_date}

def check_empty(soup):
    check_content= "There is no News according to your request."
    div_tags = soup.find_all("div", class_="item")
    return any(div_tag.find("span").text != check_content for div_tag in div_tags)

session = requests.session()
year = 2023
news_data = []

with ThreadPoolExecutor(max_workers=5) as executor:
    for month in range(6,12):
        for day in range(1,31):
            archive_url= f"https://news.rthk.hk/rthk/en/news-archive.htm?archive_year={year}&archive_month=0{month}&archive_day={day}&archive_cat=12"
            response = session.get(archive_url)
            soup = BeautifulSoup(response.content, "html.parser")
            if check_empty(soup):
                news_urls = get_url(soup)
                news_data.extend(executor.map(news_extract, news_urls))

news_df = pd.DataFrame(news_data)
news_df.to_csv(f"{year}{month:02d}Financial News from RTHK.csv")